<a href="https://colab.research.google.com/github/LinarKulinar/ml-for-sec-lab2/blob/main/Get_feature_from_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
from random import random, randrange, randint

from operator import itemgetter

import os, sys
import pandas as pd

import pickle


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA



from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score, roc_curve, plot_confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv('features/padding_level_1.0/features.csv')
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,label
0,0,2.25,25.00,2.25,25.00,2.25,361.0,2.25,361.0,0.25,...,0.00,42.25,72.25,42.25,72.25,36.0,12.25,36.0,12.25,1
1,1,20.25,12.25,20.25,12.25,2.25,49.0,2.25,49.0,72.25,...,64.00,25.00,9.00,25.00,9.00,121.0,132.25,121.0,132.25,1
2,2,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,1
3,3,0.00,0.25,0.00,0.25,16.00,9.0,16.00,9.0,0.00,...,9.00,16.00,4.00,16.00,4.00,9.0,0.25,9.0,0.25,1
4,4,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,30.25,30.25,20.25,30.25,20.25,0.0,2.25,0.0,2.25,1


In [4]:
# Разделим набор данных на независимые и зависимые переменные. Сохраняем как массив
X = df.drop('label', axis = 1)
Y = df['label']

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 21)
X_train.shape, X_test.shape

((700, 257), (300, 257))

In [8]:
# Нормализуем данные
Log = LogisticRegression()
scaler = StandardScaler()
X_train_M = scaler.fit_transform(X_train)
X_test_M = scaler.transform(X_test)

#### Логрегрессия

In [9]:
model_Log = LogisticRegression() 
model_Log.fit(X_train_M, Y_train)
Y_Log = model_Log.predict(X_test_M)

print('F1: ', f1_score(Y_test, Y_Log))
print('accuracy: ', accuracy_score(Y_test, Y_Log))

F1:  0.9965870307167235
accuracy:  0.9966666666666667


#### Метод опорных векторов

In [11]:
model_SVC = SVC()
model_SVC.fit(X_train_M, Y_train)
Y_SVC = model_SVC.predict(X_test_M)


print('F1: ', f1_score(Y_test, Y_SVC))
print('accuracy: ', accuracy_score(Y_test, Y_SVC))

F1:  0.9733333333333333
accuracy:  0.9733333333333334
